In [8]:
import pandas as pd
import ast
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

df = pd.read_csv('/Users/karth/OneDrive/Desktop/OELP/project/data.csv', delimiter=',')
df_key=pd.read_csv('/Users/karth/OneDrive/Desktop/OELP/project/keywords.csv', delimiter='\t')
name=[]
genres=[]
story=[]
rating=[]
directors=[]
pop_of_directors=[]
cast=[]
pop_of_cast=[]
for i in df['Name']:
    mve_list = ast.literal_eval(i)
    name.append(mve_list[0])
df['story'] = df['story'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stopwords.words('english')]))
for i in df['genres']:
    genres_list = ast.literal_eval(i)
    genres.append(genres_list)
for i in df['story']:
    storyline=ast.literal_eval(i)
    story.append(storyline[0].split())
for i in df['rating']:
    rating_list=ast.literal_eval(i)
    rating.append(rating_list)

d1={}
cas,rank1=[],[]
for i in df['cast']:
    i=ast.literal_eval(i)
    for j in i:
        cas.append(j)
try:
    for i in df['popularity of cast']:
        i=ast.literal_eval(i)
        for j in i:
            if j is None:
                j=5000
            rank1.append(j)
    for i in range(len(cas)):
        d1[cas[i]] = rank1[i]
except:
        for i in range(len(cas)):
            d1[cas[i]] = 0

C = []                             # M is the matrix where each row represents each movie and column represents each unique keyword
for i in df['cast']:
    x = eval(i)
    l = []
    for j in d1.keys():
        if(j in x):
            if(int(d1[j])==0):
                l.append(0.09)
            else:
                l.append(1-(int(d1[j])/11000))
        else:
            l.append(0)
    C.append(l)

d={}
dir,rank=[],[]
for i in df['directors']:
    i=ast.literal_eval(i)
    for j in i:
        dir.append(j)
for i in df['popularity of director']:
    i=ast.literal_eval(i)
    for j in i:
        rank.append(j)
for i in range(len(dir)):
    d[dir[i]] = rank[i]

R = []                             # M is the matrix where each row represents each movie and column represents each unique keyword
for i in df['directors']:
    x = eval(i)
    l = []
    for j in d.keys():
        if(j in x):
            if(int(d[j])==0):
                l.append(0.09)
            else:
                l.append(1-(int(d[j])/11000))
        else:
            l.append(0)
    R.append(l)
s = set()                           # set of all unique keywords
for i in df_key['k_h']:
    x = eval(i)
    for j in x.keys():
        s.add(j)                             
M = []                             # M is the matrix where each row represents each movie and column represents each unique keyword
for i in df_key['k_h']:
    x = eval(i)
    l = []
    for j in s:
        if(j in x.keys()):
            if(x[j]==0):
                l.append(0.5)
            else:
                l.append(int(x[j]))
        else:
            l.append(0)
    M.append(l)

d2={}
writ,rank2=[],[]
for i in df['writers']:
    i=ast.literal_eval(i)
    for j in i:
        writ.append(j)
for i in df['popularity of writer']:
    i=ast.literal_eval(i)
    for j in i:
        if j is None:
            j=5000
        rank2.append(j)
for i in range(len(writ)):
    d2[writ[i]] = rank2[i]

W = []                             # M is the matrix where each row represents each movie and column represents each unique keyword
for i in df['writers']:
    x = eval(i)
    l = []
    for j in d2.keys():
        if(j in x):
            if(int(d2[j])==0):
                l.append(0.09)
            else:
                l.append(1-(int(d2[j])/11000))
        else:
            l.append(0)
    W.append(l)

unique_genres = np.unique([genre for sublist in genres for genre in sublist])
unique_story=np.unique([sto for sublist in story for sto in sublist])

genre_matrix = [[1 if word in sublist else 0 for word in unique_genres] for sublist in genres]
key_word_matrix = np.array(M)
cast_matrix=np.array(C)
director_matrix=np.array(R)
writer_matrix=np.array(W)
story_matrix=[[1 if word in sublist else 0 for word in unique_story] for sublist in story]

def normalize_matrix(matrix, new_min, new_max):
    min_val = np.min(matrix)
    max_val = np.max(matrix)
    matrix_range = max_val - min_val

    normalized_matrix = (matrix - min_val) / matrix_range * (new_max - new_min) + new_min
    return normalized_matrix


genre_normalized_matrix = normalize_matrix(genre_matrix, 0, 1)
key_words_normalized_matrix = normalize_matrix(key_word_matrix, 0, 1)
cast_genre_normalized_matrix = normalize_matrix(cast_matrix, 0, 1)
director_normalized_matrix = normalize_matrix(director_matrix, 0, 1)
writer_normalized_matrix = normalize_matrix(writer_matrix, 0, 1)
story_normalized_matrix = normalize_matrix(story_matrix, 0, 1)

final_matrix=np.hstack((genre_normalized_matrix,key_words_normalized_matrix,cast_genre_normalized_matrix,director_normalized_matrix,writer_normalized_matrix,story_normalized_matrix))
print(len(final_matrix),len(final_matrix[0]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karth\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


1436 52884


In [9]:
from sklearn.decomposition import NMF
movie_feature_matrix = np.array(final_matrix)
def nmf_reduction(M, k):
    model = NMF(n_components=k)
    W = model.fit_transform(M)
    H = model.components_
    return W,H

In [ ]:
def perplexity(X):
    probabilities = np.exp(X)
    probabilities /= np.sum(probabilities, axis=1)[:, None]
    entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
    perplexity = np.exp(np.mean(entropy))
    return perplexity

per = {}
for i in range(100,1400,100):
    w,h = nmf_reduction(movie_feature_matrix,i)
    p = perplexity(w)
    per[p] = i
keys = list(per.keys())
k = per[min(keys)]

In [10]:
w,h = nmf_reduction(movie_feature_matrix,k)
reconstructed_matrix = np.dot(w,h)
print(w)
print(h)
cos_sim = cosine_similarity(reconstructed_matrix)

[[4.83163192e-04 0.00000000e+00 3.21673007e-03 ... 9.00805379e-03
  2.91770810e-03 1.12657182e-02]
 [6.62753931e-02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.86846327e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [7.13536369e-02 0.00000000e+00 0.00000000e+00 ... 8.10444992e-01
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.99409530e-01 0.00000000e+00 ... 4.20993552e-02
  0.00000000e+00 1.39156919e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  2.18828446e-02 0.00000000e+00]
 [5.14792046e+00 1.76890191e-01 0.00000000e+00 ... 7.26644266e-04
  1.59436991e-02 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  1.81932402e-02 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 6.31249166e-04
  0.00000000e+00 0.00000000e+00]

In [11]:
movie_name = 'Avengers: Infinity War'
try:
    index = name.index(movie_name)
    print(f"The index of {movie_name} in the list is {index}.")
except ValueError:
    print(f"{movie_name} is not present in the list.")

import numpy as np
print(index)
# Assuming you have an array named 'your_array'
max_array=np.array(cos_sim[index])
print(max_array)

# Get the indices of the 11 maximum values
max_indices = np.argsort(max_array)[:-22:-1]

# Print the indices
print("Indices of the 11 maximum values:")
print(max_indices)
inde=0
for i in max_indices:
    inde=inde+1
    print(f'{inde}-{name[i]}')

The index of Avengers: Infinity War in the list is 204.
204
[0.08033753 0.22957853 0.20550637 ... 0.07182029 0.20602352 0.04293951]
Indices of the 11 maximum values:
[ 204  469  449  501  478  127  119  255 1264  892 1110 1129  209  446
  334  386  394   90  320 1003  264]
1-Avengers: Infinity War
2-Insurgent
3-Avengers: Age of Ultron
4-Captain America: Civil War
5-Ant-Man
6-Thor: Love and Thunder
7-Avengers: Endgame
8-Thor: Ragnarok
9-The Lego Ninjago Movie
10-Serenity
11-Tides
12-Hardcore Henry
13-Justice League
14-Sonic the Hedgehog
15-Black Widow
16-Star Trek Beyond
17-X-Men: Apocalypse
18-Spider-Man: Into the Spider-Verse
19-Black Panther
20-The 5th Wave
21-Doctor Strange in the Multiverse of Madness
